In [2]:
from torch import nn #nn-neural networks
from torch.optim import Adam #our optimizer
from torch.utils.data import DataLoader #For efficient batching and parallel loading.
from torchvision import datasets  #Provides easy access to popular computer vision datasets. # Corrected import statement
import torchvision.transforms as transforms #Provides image transformations such as resizing, normalization, and flipping.
from torchvision.transforms import ToTensor #pixel range[0,1](float32) # Corrected import statement
!pip install torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 MB 13.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 29.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 28.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
dtrain = datasets.MNIST(root="data", train=True, download=True, transform = ToTensor())
dataset= DataLoader(dtrain, 32) #batch=32

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████████████████████████████████| 9.91M/9.91M [00:00<00:00, 28.4MB/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|███████████████████████████████████████| 28.9k/28.9k [00:00<00:00, 754kB/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████████████████████████████████| 1.65M/1.65M [00:00<00:00, 7.32MB/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████████████████████████████████| 4.54k/4.54k [00:00<00:00, 1.48MB/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [4]:
class ImageClassifier(nn.Module):
  def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, (3,3)), # ((28-3+2*0)/1 )+1
            nn.ReLU(),
            nn.Conv2d(32, 64, (3,3)), #((26-3+2*0)/1 )+1
            nn.ReLU(),
            nn.Conv2d(64, 64, (3,3)), #((24-3+2*0)/1 )+1
            nn.ReLU(),
            nn.Flatten(), 
            nn.Linear(64*(28-6)*(28-6), 10)  
        )
        
  def forward(self, x): 
    return self.model(x)

In [5]:
import torch
import torch.nn as nn
from torch.optim import Adam
# Check device (MPS for Mac, CPU otherwise)
device = "mps" if torch.backends.mps.is_available() else "cpu"

# Model, optimizer, and loss function
clf = ImageClassifier().to(device)
opt = Adam(clf.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()


In [7]:

from torch import save, load
from PIL import Image
# Create model and move it to device
clf = ImageClassifier().to(device)
opt = Adam(clf.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

# Training loop
if __name__ == "__main__":
    for epoch in range(10):  # Train for 10 epochs
        for batch in dataset:
            X, y = batch        #unpack data
            X, y = X.to(device), y.to(device)  # Move data to GPU 
            yhat = clf(X)      #prediction 
            loss = loss_fn(yhat, y)

            # Apply backpropagation
            opt.zero_grad()   #zero out any existing gradients 
            loss.backward()
            opt.step()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")  #print loss for every epoch 

    # Save model state
    with open('model_state.pt', 'wb') as f:
        save(clf.state_dict(), f)

    # Load model state
    with open('model_state.pt', 'rb') as f:
        clf.load_state_dict(load(f, map_location=device))  # Ensure it's loaded on the correct device

    # Load and preprocess an image
    img = Image.open('img_3.jpg')
    img_tensor = ToTensor()(img).unsqueeze(0).to(device)  # Move to MPS or CPU

    # Make a prediction
    print(torch.argmax(clf(img_tensor)))


Epoch: 0, Loss: 0.013781211338937283
Epoch: 1, Loss: 0.003175342921167612
Epoch: 2, Loss: 0.0001682673319010064
Epoch: 3, Loss: 0.0018442970467731357
Epoch: 4, Loss: 2.199448863393627e-05
Epoch: 5, Loss: 4.384481599117862e-06
Epoch: 6, Loss: 5.625167318612512e-07
Epoch: 7, Loss: 2.607703031287656e-08
Epoch: 8, Loss: 1.4359444321598858e-05
Epoch: 9, Loss: 6.734931048413273e-06


/var/folders/xz/6ncw4lx92xgf7_wfqbgtdqj00000gn/T/ipykernel_25263/478920045.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  clf.load_state_dict(load(f, map_location=devi

tensor(9, device='mps:0')
